In [1]:
!pip install opencv-python

In [2]:
import sys
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,AveragePooling2D,Dense,Dropout,Activation,Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing import image

E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [3]:
data = pd.read_csv('fer2013.csv')

In [4]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
x_train,y_train,x_test,y_test=[],[],[],[]

In [6]:
for index,row in data.iterrows():
  val = row['pixels'].split(' ')
  try:
    if 'Training' in row['Usage']:
      x_train.append(np.array(val,'float32'))
      y_train.append(row['emotion'])
    elif 'PublicTest' in row['Usage']:
      x_test.append(np.array(val,'float32'))
      y_test.append(row['emotion'])
  except:
        print('',end = '')

print(f"x_train sample data:{x_train[0:2]}")
print(f'y_train sample data:{y_train[0:2]}')
print(f'x_test sample data:{x_test[0:2]}')
print(f'y_test sample data:{y_test[0:2]}')

num_features = 64
num_labels = 7
batch_size = 64
epochs = 115
width, height = 48, 48


x_train = np.array(x_train,'float32')
y_train = np.array(y_train,'float32')
x_test = np.array(x_test,'float32')
y_test = np.array(y_test,'float32')

y_train = to_categorical(y_train, num_classes=num_labels, dtype='float32')
y_test = to_categorical(y_test, num_classes=num_labels, dtype='float32')

x_train -= np.mean(x_train, axis=0)
x_train /= np.std(x_train, axis=0)

x_test -= np.mean(x_test, axis=0)
x_test /= np.std(x_test, axis=0)

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)

x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)

#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(x_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

x_train sample data:[array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32), array([151., 150., 147., ..., 193., 183., 184.], dtype=float32)]
y_train sample data:[0, 0]
x_test sample data:[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32), array([156., 184., 198., ..., 172., 167., 161.], dtype=float32)]
y_test sample data:[0, 1]
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
#batch process
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)


In [8]:
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
model.compile(loss='categorical_crossentropy'
    , optimizer=Adam()
    , metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

Train on 28709 samples, validate on 3589 samples
Epoch 1/115
28709/28709 [==============================] - 180s 6ms/sample - loss: 1.7047 - acc: 0.3043 - val_loss: 1.5267 - val_acc: 0.4076
Epoch 2/115
28709/28709 [==============================] - 179s 6ms/sample - loss: 1.4918 - acc: 0.4180 - val_loss: 1.3825 - val_acc: 0.4564
Epoch 3/115
28709/28709 [==============================] - 181s 6ms/sample - loss: 1.3848 - acc: 0.4655 - val_loss: 1.3201 - val_acc: 0.4898
Epoch 4/115
28709/28709 [==============================] - 178s 6ms/sample - loss: 1.3253 - acc: 0.4895 - val_loss: 1.3427 - val_acc: 0.4887
Epoch 5/115
28709/28709 [==============================] - 177s 6ms/sample - loss: 1.2768 - acc: 0.5096 - val_loss: 1.2382 - val_acc: 0.5241
Epoch 6/115
28709/28709 [==============================] - 179s 6ms/sample - loss: 1.2433 - acc: 0.5265 - val_loss: 1.2141 - val_acc: 0.5300
Epoch 7/115
28709/28709 [==============================] - 177s 6ms/sample - loss: 1.2139 - acc: 0.5359 -

28709/28709 [==============================] - 177s 6ms/sample - loss: 0.6043 - acc: 0.7776 - val_loss: 1.3956 - val_acc: 0.5720
Epoch 59/115
28709/28709 [==============================] - 176s 6ms/sample - loss: 0.6044 - acc: 0.7777 - val_loss: 1.4039 - val_acc: 0.5720
Epoch 60/115
28709/28709 [==============================] - 178s 6ms/sample - loss: 0.5870 - acc: 0.7853 - val_loss: 1.4067 - val_acc: 0.5723
Epoch 61/115
28709/28709 [==============================] - 179s 6ms/sample - loss: 0.5943 - acc: 0.7814 - val_loss: 1.4861 - val_acc: 0.5717
Epoch 62/115
28709/28709 [==============================] - 177s 6ms/sample - loss: 0.5726 - acc: 0.7910 - val_loss: 1.4014 - val_acc: 0.5809
Epoch 63/115
28709/28709 [==============================] - 182s 6ms/sample - loss: 0.5755 - acc: 0.7886 - val_loss: 1.4706 - val_acc: 0.5756
Epoch 64/115
28709/28709 [==============================] - 180s 6ms/sample - loss: 0.5722 - acc: 0.7929 - val_loss: 1.4221 - val_acc: 0.5756
Epoch 65/115
28709/

28709/28709 [==============================] - 198s 7ms/sample - loss: 0.3938 - acc: 0.8620 - val_loss: 1.8181 - val_acc: 0.5592


In [10]:
fer_json = model.to_json()
with open('fer.json','w') as json_file:
  json_file.write(fer_json)

model.save_weights("fer.h5")

In [11]:
#load model
model = model_from_json(open("fer.json", "r").read())
#load weights
model.load_weights('fer.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [12]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [13]:
cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>